# Add Agency and Agency Information to FMIS IIJA

In [1]:
import numpy as np
import pandas as pd
from siuba import *

from shared_utils import geography_utils
from dla_utils import _dla_utils

from calitp import to_snakecase

import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)


In [3]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

## Read In Data

In [105]:
## other county locodes

# county = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/Copy of County.xlsx",
#                                    sheet_name='locode1',
#                                    header=[0]))

In [6]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/CopyofFMIS_Projects_Universe_IIJA_Reporting_4.xls", 
                           # sheet_name='FMIS 5 Projects  ', header=[3]
                           sheet_name='IIJA-combined',
                           # sheet_name='FMIS 5 Projects  ',
                           ))

In [9]:
# number of entries in the county locode list
len(county)

1072

In [10]:
county.sample(4)

agency_locode                         agency_name  district      mpo  \
859            6312                     Access Services      7.00     SCAG   
1067           7511  Santa Ynez Band of Chumash Indians      5.00  NON-MPO   
787            6240              Napa Valley Wine Train      4.00      MTC   
772            6225  El Dorado County Transit Authority      3.00    SACOG   

                    county  
859     Los Angeles County  
1067  Santa Barbara County  
787            Napa County  
772       El Dorado County

In [12]:
proj.drop(columns =['unnamed:_0', 'unnamed:_13', 'total'], axis=1, inplace=True)

In [13]:
proj.sample()

fmis_transaction_date program_code    program_code_description  \
94            2022-08-11         Y400  CONGESTION MITIGATION IIJA   

   project_number recipient_project_number  \
94        6164024              0822000089L   

                                                                                                                                            project_title  \
94  EASTERN RIVERSIDE COUNTY (18 CORRIDORS): MONTEREY, COOK, PALM DR, BOB HOPE, FRED WARING, DINAH SHORE, GENE AUTRY, DATE PALM, INDIO BLVD, JEFFERSON, P   

    county_code congressional_district improvement_type  \
94           65           Cong Dist 36                1   

   improvement_type_description  obligations_amount  \
94    New  Construction Roadway       30,886,000.00   

   summary_recipient_defined_text_field_1_value  
94                                    L6164SCAG

## Get Locode Substring

In [14]:
string = proj['summary_recipient_defined_text_field_1_value'].iloc[0]

In [15]:
string

'L5253SCAG'

In [16]:
print(string.find('5'))
print(string.find('3'))

1
4


In [17]:
# need to extract string from position 1-4. 

In [18]:
#proj['locode'] = proj.summary_recipient_defined_text_field_1_value.apply(lambda x: x[1:5])

In [19]:
#proj.head()

In [20]:
county.sample()

agency_locode                                agency_name  district  mpo  \
776           6229  Hayward Area Recreation And Park District      4.00  MTC   

             county  
776  Alameda County

## Merge data on Locode

In [21]:
proj_all = utils.add_name_from_locode(proj, 'summary_recipient_defined_text_field_1_value')

In [24]:
proj_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 18 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fmis_transaction_date                         330 non-null    datetime64[ns]
 1   program_code                                  330 non-null    object        
 2   program_code_description                      330 non-null    object        
 3   project_number                                330 non-null    object        
 4   recipient_project_number                      330 non-null    object        
 5   project_title                                 330 non-null    object        
 6   county_code                                   330 non-null    int64         
 7   congressional_district                        330 non-null    object        
 8   improvement_type                              330 non-null    object  

In [28]:
# one locode did not match
proj_all>>filter(_.implementing_agency_locode.isnull())

fmis_transaction_date program_code    program_code_description  \
284            2022-04-26         Y001  NATIONAL HIGHWAY PERF IIJA   

    project_number recipient_project_number  \
284        NBIS522             0000001453L5   

                                                                                           project_title  \
284  STATEWIDE - IN SERVICE BRIDGES OWNED BY LOCAL AGENCIES FEDERALLY MANDATED BRIDGE INSPECTION PROGRAM   

     county_code congressional_district improvement_type  \
284           67            Cong Dist 3               49   

                      improvement_type_description  obligations_amount  \
284  Bridge Inspection and Bridge Related Training       14,164,800.00   

    summary_recipient_defined_text_field_1_value  implementing_agency_locode  \
284                                 S    NON-MPO                         NaN   

    implementing_agency  district county_name rtpa_name mpo_name  
284                 NaN       NaN         NaN       NaN      NaN

In [30]:
list(proj_all.implementing_agency.unique())

['Hawthorne',
 'Yucaipa',
 'Fairfield',
 'Lake County',
 'Vacaville',
 'Benicia',
 'Pico Rivera',
 'Marin County',
 'Los Banos',
 'Santa Clara County',
 'Santa Barbara County',
 'Fresno County',
 'Santa Ana',
 'Stockton',
 'Caltrans',
 'Sacramento',
 'San Benito County',
 'Pinole',
 'Sacramento County',
 'Merced County',
 'Shasta County',
 'Livingston',
 'El Dorado County',
 'Sanger',
 'Palmdale',
 'Tulare County',
 'Coachella',
 'Bakersfield',
 'Colton',
 'Visalia',
 'Rialto',
 'Metropolitan Transportation Commission',
 'Bellflower',
 'Belmont',
 'Larkspur',
 'Monterey County',
 'Madera County',
 'Moraga',
 'Orinda',
 'Redding',
 'Cathedral City',
 'Alameda',
 'Clovis',
 'Gardena',
 'Mission Viejo',
 'Pleasanton',
 'Fresno',
 'Butte County',
 'Rohnert Park',
 'Alameda County Transportation Commission',
 'Yolo County',
 'Stanislaus County',
 'Placer County',
 'Contra Costa County',
 'Mariposa County',
 'Santa Barbara',
 'Nevada County',
 'Calaveras County',
 'Long Beach',
 'Los Angeles

## Writing to GCS

In [31]:
#proj_all.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_wip.csv")

## Adding Place Names (can do)
* we have a list of city names to county from [Caltrans PlaceNames](https://dot.ca.gov/-/media/dot-media/programs/research-innovation-system-information/documents/place-names/2019-place-names-in-california-a11y.pdf)


In [32]:
# city_place_names = (to_snakecase(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/2020-place-names-locode.xlsx', sheet_name=0)))

In [33]:
# city_place_names.sample()

In [34]:
# city_place_names.drop(columns =['unnamed:_1', 'unnamed:_3', 'unnamed:_4','unnamed:_6','unnamed:_7', 'date_of_incorporation',
#                                'city_name_abbr_','name','dist_', 'co_'], axis=1, inplace=True)

In [35]:
# (pd.merge(proj_all, city_place_names, left_on='agency_locode', right_on='ct_city_code', how='left', indicator=True))._merge.value_counts()

In [36]:
# proj_all1 = (pd.merge(proj_all, city_place_names, left_on='agency_locode', right_on='ct_city_code', how='left', indicator='City'))

In [37]:
# proj_all1.sample(2)

In [38]:
# proj_all1>>filter(_.City!='both')

In [39]:
# county_place_names = (to_snakecase(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/2020-place-names-locode.xlsx', sheet_name=1)))

In [40]:
# county_place_names

## Project Location

In [41]:
location = (proj_all>>select(_.implementing_agency, _.county_name, _.project_title))

In [42]:
location.sample()

implementing_agency    county_name  \
256       Sierra County  Sierra County   

                                                                                                 project_title  
256  SALMON LAKE ROAD AT SALMON CREEK ~ 300 FT WEST OF GOLD LAKE HIGHWAY. BR.# 13C0053 BRIDGE REPLACEMENT (TC)

In [43]:
len(location)

330

## Project Title

In [44]:
#phrase extraction maybe: https://stackoverflow.com/questions/70995812/extract-keyword-from-sentences-in-a-pandas-text-column-using-nltk-and-or-regex

In [45]:
# ! pip install nltk
# ! pip install textblob

In [46]:
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [47]:

import re

In [48]:
#subset df to work on title
title = proj_all>>select(_.implementing_agency, _.program_code_description, _.project_title, _.program_code)

In [49]:
title

implementing_agency       program_code_description  \
0             Hawthorne   HIGHWAY SAFETY IMP PROG IIJA   
1               Yucaipa     NATIONAL HIGHWAY PERF IIJA   
2             Fairfield   HIGHWAY SAFETY IMP PROG IIJA   
3           Lake County    STBG IIJA OFF-SYSTEM BRIDGE   
4             Vacaville   HIGHWAY SAFETY IMP PROG IIJA   
..                  ...                            ...   
325               Indio     NATIONAL HIGHWAY PERF IIJA   
326           Kingsburg     CONGESTION MITIGATION IIJA   
327           Santa Ana  TRANS ALTERNATIVES >200K IIJA   
328  San Joaquin County    STBG IIJA OFF-SYSTEM BRIDGE   
329            Woodlake     CONGESTION MITIGATION IIJA   

                                                                                                                                             project_title  \
0           120TH STREET FROM PRAIRIE AVENUE TO FELTON AVENUE. TRAFFIC SIGNAL UPGRADES AT NINE SIGNALIZED INTERSECTIONS AND IMPROVE CROSSINGS AND SIGNAGE.   
1                           13TH STREET OVER WILSON CREEK FROM OAK GLEN ROAD TO KENTUCKY STREET, LWC 00L0017 REPLACE LOW WATER CROSSING WITH 2-LANE BRIDGE   
2         16 LOCATIONS IN THE CITY OF FAIRFIELD INSTALL ADAPTIVE SIGNAL TIMING AND ADVANCED DILEMMA-ZONE DETECTION, AND ENHANCE THE VISIBILITY OF SIGNALS.   
3                                         2.2 MILES NORTH OF STATE ROUTE 20 ON WITTER SPRINGS ROAD OVER COOPER CREEK. BR.# 14C0119 BRIDGE REPLACEMENT (TC)   
4    21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR   
..                                                                                                                                                     ...   
325                                           WESTBOUND INDIO BOULEVARD OVER WHITEWATER RIVER, BR. NO. 56C-0292 SEISMIC RETROFIT AND SCOUR COUNTERMEASURES   
326                                                                             WESTSIDE OF 18TH AVE FROM STROUD AVE TO KLEPPER ST CONSTRUCT NEW SIDEWALKS   
327    WILLITS STREET FROM FAIRVIEW STREET TO E/S OF RAITT STREET INSTALL MEDIAN, PARKING-PROTECTED BICYCLE LANES, AND DEDICATED BICYCLE SIGNAL HEADS (TC)   
328                                                                     WIMER ROAD OVER INDIAN CREEK NORTH BRANCH (BRIDGE 29C0303) BRIDGE REPLACEMENT (TC)   
329                                                                  WOODLAKE AT THE INTERSECTION OF SIERRA AVE AND CASTLE ROCK STREET NEW ROUNDABOUT (TC)   

               program_code  
0                YS30//YS30  
1                      Y001  
2                YS30//YS30  
3    Y233//Y233//Y233//Y233  
4                YS30//YS30  
..                      ...  
325              Y001//Y001  
326              Y400//Y400  
327              Y301//Y301  
328              Y233//Y233  
329                    Y400  

[330 rows x 4 columns]

In [50]:
title.program_code_description.value_counts()

HIGHWAY SAFETY IMP PROG IIJA      75
STBG-URBANIZED >200K IIJA         61
NATIONAL HIGHWAY PERF IIJA        59
STBG IIJA OFF-SYSTEM BRIDGE       49
CONGESTION MITIGATION IIJA        37
SURFAC TRNSP BLK GRTS-FLX IIJA    18
TRANS ALTERNATIVES >200K IIJA     14
PROJ TO REDUCE PM 2.5 EMI IIJA    12
TRANSP ALTERNATIVES FLEX IIJA      4
TRANS ALTERN 50K-200K POP IIJA     1
Name: program_code_description, dtype: int64

In [52]:
clean_type_list_df = utils.get_list_of_words(title, "project_title")

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
clean_type_list_df.value_counts()

bridge     178
road       142
avenue     115
ave         77
street      75
          ... 
jamacha      1
bellota      1
belmont      1
j            1
00           1
Length: 1729, dtype: int64

In [54]:
clean_type_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4910 entries, 0 to 4909
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4910 non-null   object
dtypes: object(1)
memory usage: 38.5+ KB


In [55]:
clean_type_list_df.value_counts()

bridge     178
road       142
avenue     115
ave         77
street      75
          ... 
jamacha      1
bellota      1
belmont      1
j            1
00           1
Length: 1729, dtype: int64

In [56]:
#splitting up the types of words based on the most common words appearing in title name

fix_type = ['REPLACEMENT', 'INSTALL', 'CONSTRUCT', 'REPLACE', 'SIGNAL', 'TRAFFIC',
           'IMPROVEMENT', 'PEDESTRIAN', 'LANES', 'NEW', 'REHABILITATION',
           'UPGRADE', 'CLASS', 'BIKE', 'WIDEN', 'LANDSCAPING', 'SAFETY', 'RAISED', 
            'SEISMIC', 'SIGNAGE', 'RETROFIT', 'ADD', 'PLANNING', 'PAVE'
           'PREVENTIVE','MAINTENANCE', 'REHAB', 'RESURFACE', 'REPAIR', 'ROUNDABOUT']

area_type = ['BRIDGE', 'ROAD', 'RD', 'AVENUE', 'AVE', 'STREET' , 'ST',
             'FRACTURED', 'LANE', 'DRIVE', 'boulevard', 'BLVD',
             'INTERSECTION', 'intersections', 'WAY', 'DR', 'CURB', 'ROADWAY',
             'TRAIL', 'PATH', 'CREEK', 'RIVER', 'SIDEWALK', 'CORRIDOR', 'PARKWAY',
            'RAMPS', 'GUARDRAIL']

jurisdiction = ['CITY', 'COUNTY', 'STATE', 'UNINCORPORATED']

other = ['TC', 'EXISTING']

In [57]:
def tokenize(texts):
    return [nltk.tokenize.word_tokenize(t) for t in texts]

### using  np.concatenate

In [58]:
#! pip install more-itertools

In [59]:
#from more_itertools import split_after

In [61]:
title.sample()

implementing_agency      program_code_description  \
289             Benicia  HIGHWAY SAFETY IMP PROG IIJA   

                                                                                                                                             project_title  \
289  THE INTERSECTIONS OF MILITARY WEST AND WEST 5TH ST. AND HASTINGS DR. AND LONDON DR. PEDESTRIAN CROSSING IMPROVEMENTS, INCLUDING: ADA RAMPS RRFB SYSTE   

    program_code  
289   YS30//YS30

In [62]:
text = title>>select (_.project_title)

In [63]:
text['summary'] = utils.key_word_intersection(text, 'project_title')

In [64]:
text.sample(10)

project_title  \
253  SACOG REGION - IN SACRAMENTO, SUTTER, YOLO AND YUBA COUNTIES CONDUCT NECESSARY PLANNING, STATE AND FEDERAL PROGRAMMING, AND MONITORING ACTIVITIES, IN   
178                         JENSEN AVE. FROM ELM AVENUE TO TEMPERANCE AVENUE INSTALL ITS ADAPTIVE SYSTEM, UPGRADE DETECTION AND SYNCHRONIZE CORRIDOR. (TC)   
189                      MATTOLE ROAD OVER MATTOLE RIVER (HONEYDEW), EAST OF WILDER RIDGE ROAD AND.MATTOLE ROAD JUNCTION BRIDGE REPLACEMENT (BR # 04C0055)   
255                                             SALEM STREET AT LITTLE CHICO CREEK BRIDGE BR. # 12C-336 ~0.1 M NORTH OF 10TH ST BRIDGE REHABILITATION (TC)   
6     3/8 MI WEST OF PARAMOUNT BLVDWASHINGTON BLVD OVER RIO HONDO.LOCAL BRIDGE # 53C0471 BRIDGE REPLACEMENT  (DEMOLISH THE EXISTING 6 LANNE SUPERSTRUCTURE   
52                                                   BRIDGE NO. 28C0331, BEAR CREEK RD, OVER SAN PABLO CREEK, 0.2 MI EAST OF CAMINO PABLO SEISMIC RETROFIT   
247                                                               REQUA ROAD OVER HUNTER CREEK, 0.1 MI. WEST OF SR-101 REPLACE EXISTING 2-LANE BRIDGE (TC)   
312                                                               VARIOUS LOCATIONS THROUGHOUT THE WEST SIDE OF THE CITY. REPAIR EXISTING DAMAGED GUARDRAI   
289  THE INTERSECTIONS OF MILITARY WEST AND WEST 5TH ST. AND HASTINGS DR. AND LONDON DR. PEDESTRIAN CROSSING IMPROVEMENTS, INCLUDING: ADA RAMPS RRFB SYSTE   
170  INTERSECTIONS OF 65TH ST EXPWY AT 21ST AVE, 65TH ST EXPWY AT FRUITRIDGE RD, ARDEN WAY AT HERITAGE LN, ARDEN WAY AT CHALLENGE WAY, HOWE AVE. AT SWARTH   

                              summary  
253                 [PLANNING, STATE]  
178                [INSTALL, UPGRADE]  
189                          [BRIDGE]  
255  [BRIDGE, BRIDGE, REHABILITATION]  
6                    [BRIDGE, BRIDGE]  
52        [BRIDGE, SEISMIC, RETROFIT]  
247                 [REPLACE, BRIDGE]  
312                    [CITY, REPAIR]  
289       [INTERSECTIONS, PEDESTRIAN]  
170                   [INTERSECTIONS]

In [65]:
text>>filter(_.project_title.str.contains('COMPLETE STREET'))

project_title  \
23   ARDEN WAY FROM ETHAN WAY TO MORSE AVENUE.  ARDEN WAY, FROM FULTON AVENUE TO MORSE AVENUE COMPLETE STREET IMPROVEMENTS INCLUDING SEPARATED SIDEWALK, C   
29         AVE R  BETWEEN SIERRA HIGHWAY AND 25TH STREET. AVE R COMPLETE STREET WITH SIDEWALKS GAP CLOSURES, BIKE LANES, ADA RAMPS AND ENHANCED CROSSWALKS   
99                                                                            EL CAMINO REAL (SR82) FROM ARROYO TO KAISER WAY COMPLETE STREET IMPROVEMENTS   
129                                                               HEALDSBURG AVENUE BETWEEN POWELL AVENUE AND PASSALAQUA ROAD COMPLETE STREET IMPROVEMENTS   
290   THE TOWNSITE DRIVE COMPLETE STREETS PROJECT IS LOCATED IN THE CITY OF VISTA ALONG THE SEGMENT OF NORTH DRIVE FROM JUST NORTHWEST OF THE INTERSECTION   

                  summary  
23             [SIDEWALK]  
29          [BIKE, LANES]  
99                     []  
129                    []  
290  [CITY, INTERSECTION]

In [66]:

text>>filter(_.project_title.str.contains('BRIDGE REPLACEMENT'))

project_title  \
3                                         2.2 MILES NORTH OF STATE ROUTE 20 ON WITTER SPRINGS ROAD OVER COOPER CREEK. BR.# 14C0119 BRIDGE REPLACEMENT (TC)   
6     3/8 MI WEST OF PARAMOUNT BLVDWASHINGTON BLVD OVER RIO HONDO.LOCAL BRIDGE # 53C0471 BRIDGE REPLACEMENT  (DEMOLISH THE EXISTING 6 LANNE SUPERSTRUCTURE   
10                                                                         ALAMITOS RD. BRIDGE @ ALAMITOS CK (37C0159) BRIDGE REPLACEMENT/SEISMIC RET (TC)   
24                                                                            ARROYA AVENUE OVER WEST DELTA CANAL (BRIDGE 39C0275) BRIDGE REPLACEMENT (TC)   
25                                                                          ASH CREEK ROAD AT SACRAMENTO RIVER OVERFLOW, BRIDGE 06C0233 BRIDGE REPLACEMENT   
41                                                                    BON AIR BRIDGE (BON AIR RD OVER CORTE MADERA CREEK). BR.# 27C0028 BRIDGE REPLACEMENT   
53                                                                                           BUCK AVENUE OVER ALAMO CREEKBR NO. 23C0011 BRIDGE REPLACEMENT   
54                                                                                                CANYON RD @ ACID CANAL. BR. # 06C0307 BRIDGE REPLACEMENT   
55                                                                            CASSEL FALL RIVER ROAD BRIDGE AT PIT RIVER. BR. # 06C0039 BRIDGE REPLACEMENT   
68          CR 40 OVER CACHE CREEK, 0.12 MILES SOUTH OF SR 16 BRIDGE REPLACEMENT. REPLACE EXISTING 1 LANE BRIDGE WITH A NEW 1 LANE BRIDGE. BR#22C0091 (TC)   
69                                                                                   CRABTREE ROAD OVER DRY CREEK (BRIDGE 38C0009) BRIDGE REPLACEMENT (TC)   
73                                                                                   DARRAH ROAD OVER CHOWCHILLA RIVER (BRIDGE 40C0033) BRIDGE REPLACEMENT   
74                                            DE LA GUERRA ST AT MISSION CREEK IN SANTA BARBARA BRIDGE REPLACEMENT - ENVIRONMENTAL MITIGATION & MONITORING   
77                                                                                 DOGTOWN ROAD OVER INDIAN CREEK (BRIDGE 30C0051) BRIDGE REPLACEMENT (TC)   
78                                                                                DOGTOWN ROAD OVER MAXWELL CREEK (BRIDGE 40C0038) BRIDGE REPLACEMENT (TC)   
79                                                                                DOGTOWN ROAD OVER MAXWELL CREEK (BRIDGE 40C0039) BRIDGE REPLACEMENT (TC)   
89                                                   EAST RIO BONITO RD AT SUTTER BUTTES CANAL 0.8 MI EAST OF SR99 - BR. #12C-0165 BRIDGE REPLACEMENT (TC)   
100                                                                            ESCALON BELLOTA ROAD OVER MORMON SLOUGH (BRIDGE 29C0051) BRIDGE REPLACEMENT   
102                                                           FERNALD PT. LN. BRIDGE #51C0137 @ ROMERO CRK IN SANTA BARBARA COUNTY BRIDGE REPLACEMENT (TC)   
117                                                                                            FRANKLIN BLVD AT LOST SLOUGH BR# 24C0155 BRIDGE REPLACEMENT   
121                                                     FRENCH BAR ROAD (BRIDGE 26C0035) OVER SF JACKSON CREEK, .1 MILES E/O SR 49 BRIDGE REPLACEMENT (TC)   
123                                           GAS POINT ROAD BRIDGE (06C-0183) OVER  NO NAME DITCH, 1.9 MILES WEST OF I-5. BR.# 06C0183 BRIDGE REPLACEMENT   
128                                                                                                           HARTNELL ROAD BRIDGE BRIDGE REPLACEMENT (TC)   
154                                                                                              IN SONOMA COUNTY AT WATMAUGH RD BRIDGE BRIDGE REPLACEMENT   
175                                                     IOWA CITY ROAD AT JACK SLOUGH BR. # 16C0077 ~0.3 MI. EAST OF FRUITLAND RD. BRIDGE REPLACEMENT (TC)   
186                              

In [67]:
text>>filter(_.project_title.str.contains('WIDEN'))

project_title  \
81    DOWNTOWN LOS ANGELES, BROADWAY BETWEEN 4TH AND 6TH STREETS PEDESTRIAN SAFETY IMPROVEMENTS INCLUDING CURB EXTENSIONS, WIDENED SIDEWALK, CROSSWALK AND   
104  FIRST STREET BETWEEN FLOWER ST AND STANDARD AVE WIDEN EXISTING SIDEWALKS BY 3FT, NARROW VEHICLE LANES, AND CONSTRUCT ADA IMPROVEMENTS ON SIDEWALKS AN   
156  IN STOCKTON, PARALLEL TO MARCH LANE IN THE EAST BAY MUD CORRIDOR BETWEEN BROOKSIDE ROAD AND HILLSBORO WAY. RECONSTRUCT, WIDEN AND IMPROVE EXISTING PA   
158  IN THE CITY OF GALT ON KOST ROAD AT UNION PACIFIC RAILROAD CROSSING, WEST OF JOY DR., EAST OF MARIA WAY. WIDEN 400 FEET OF KOST RD. ON EACH SIDE OF T   
181                                                                             LA PAZ ROAD: MUIRLANDS BLVD. TO CRISANTA DR. WIDEN TWO OVERHEADS OVER BNSF   
194                                                                          MILLERTON ROAD FROM FRIANT ROAD TO MARINA DRIVE WIDEN ROADWAY FROM 2LU TO 4LD   
204  NEES AVENUE FROM MINNEWAWA AVENUE TO CLOVIS AVENUE ROAD WIDENING AND RECONSTRUCTION, INSTALLATION OF CURB AND GUTTERS, SIDEWALK, BICYCLE LANES, MODIF   
220                                        ON OLD OREGON TRAIL BETWEEN PASO ROBLES AVENUE AND BEAR MOUNTAIN ROAD WIDEN SHOULDERS AND UPGRADE DRAINAGE (TC)   
233  PARK ROAD  FROM 250 FT SOUTH OF OAK ROAD TO BAYSHORE ROAD REHAB./RESURFACE  ROADWAY AND WIDEN ROAD TO INSTALL ASPHALT CONCRETE PAVED CLASS II/V BICYC   
241   POWER INN ROAD FROM ELSIE AVENUE TO ABOUT 400 FEET NORTH OF MACFADDEN DRIVE. INSTALL CURB, GUTTER, SIDEWALK INFILL AND CURB RAMPS; WIDEN SUBSTANDARD   
244                                  RAMON ROAD FROM SAN LUIS REY DRIVE TO LANDAU BOULEVARD, BR. NO. 56C-0287 REHABILITATE AND WIDEN BRIDGE AND APPROACHES   
249  RIVERLAND DRIVE FROM KNIGHTON RD TO 2 MILES SOUTH IN THE UNINCORPORATED AREA OF SHASTA COUNTY WIDEN EXISTING ROADWAY BY PAVING SHOULDERS TO ACCOMMODA   
280  SR-51 (CAPITAL CITY) CORRIDOR IMPROVEMENTS FROM J ST TO ARDEN WAY. WIDEN AND CONSTRUCT BUS/CARPOOL LANES, TRANSITION/AUXILIARY & NB CLASS I BIKE PATH   
293  TULARE STREET FROM SIXTH STREET TO CEDAR AVENUE. INSTALL AND WIDEN SIDEWALKS ALONG THE SOUTH SIDE OF TULARE STREET, UPGRADE CURB RAMPS, RE-STRIPE ROA   
296  US-395 (HESPERIA, VICTORVILLE, & ADELANTO) FROM 0.16 MI N/O INTERSTATE ROUTE 15 JUNCTION TO SR18 WIDEN FROM 2-4 LANES AND ADD LEFT TURN CHANNELIZATIO   

                                          summary  
81                 [PEDESTRIAN, SAFETY, SIDEWALK]  
104                     [WIDEN, LANES, CONSTRUCT]  
156                                       [WIDEN]  
158                                 [CITY, WIDEN]  
181                                       [WIDEN]  
194                                       [WIDEN]  
204                             [SIDEWALK, LANES]  
220                              [WIDEN, UPGRADE]  
233                       [WIDEN, INSTALL, CLASS]  
241                    [INSTALL, SIDEWALK, WIDEN]  
244                               [WIDEN, BRIDGE]  
249               [UNINCORPORATED, COUNTY, WIDEN]  
280  [CITY, WIDEN, CONSTRUCT, LANES, CLASS, BIKE]  
293                     [INSTALL, WIDEN, UPGRADE]  
296                           [WIDEN, LANES, ADD]

In [68]:
type(text['summary'])

pandas.core.series.Series

### using if statement

In [69]:
title.sample()

implementing_agency      program_code_description  \
59             Gardena  HIGHWAY SAFETY IMP PROG IIJA   

                                                                                                                                            project_title  \
59  CITY OF GARDENA, VERMONT AVENUE JUST NORTH OF 133RD STREET. INSTALL A PEDESTRIAN CROSSWALK WITH A PEDESTRIAN HYBRID BEACON (PHB) OR HIGH-INTENSITY AC   

   program_code  
59   YS30//YS30

In [70]:
type_list = ['BRIDGE REPLACEMENT', 'COMPLETE STREET', 'VIDEO DETECTION EQUIPMENT', 'SYNCHRONIZE CORRIDOR', 'ROADWAY REALIGNMENTS']

In [71]:
text>>filter(
            _.project_title.str.contains('UPGRADE')| _.project_title.str.contains('IMPROVE')
            )

project_title  \
0           120TH STREET FROM PRAIRIE AVENUE TO FELTON AVENUE. TRAFFIC SIGNAL UPGRADES AT NINE SIGNALIZED INTERSECTIONS AND IMPROVE CROSSINGS AND SIGNAGE.   
4    21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR   
5                                                                          21 LOCATIONS IN CITY OF BENICIA UPGRADE EXISTING GUARDRAILS AND END TREATMENTS.   
20                                                                               AMERICAN AVENUE FROM MADERA AVENUE TO PLACER AVENUE SHOULDER IMPROVEMENTS   
22   APPIAN WAY AND MARLESTA ROAD INTERSECTION SAFETY IMPROVEMENTS: INSTALLATION OF A TRAFFIC SIGNAL AND TRAFFIC SIGNAGE, INCLUDING ADVANCED WARNING SIGNS   
23   ARDEN WAY FROM ETHAN WAY TO MORSE AVENUE.  ARDEN WAY, FROM FULTON AVENUE TO MORSE AVENUE COMPLETE STREET IMPROVEMENTS INCLUDING SEPARATED SIDEWALK, C   
26                                                                   AT MAX FOSTER SPORTS COMPLEX AREA IN LIVINGSTON CONSTRUCT MULTI-USE PATH IMPROVEMENTS   
81    DOWNTOWN LOS ANGELES, BROADWAY BETWEEN 4TH AND 6TH STREETS PEDESTRIAN SAFETY IMPROVEMENTS INCLUDING CURB EXTENSIONS, WIDENED SIDEWALK, CROSSWALK AND   
85                                                        E. 20TH STREET FROM THE MALL TO THE END OF BUSINESS LANE (ADJACENT TO SR99) BIKEWAY IMPROVEMENTS   
99                                                                            EL CAMINO REAL (SR82) FROM ARROYO TO KAISER WAY COMPLETE STREET IMPROVEMENTS   
104  FIRST STREET BETWEEN FLOWER ST AND STANDARD AVE WIDEN EXISTING SIDEWALKS BY 3FT, NARROW VEHICLE LANES, AND CONSTRUCT ADA IMPROVEMENTS ON SIDEWALKS AN   
116                                                                        FOWLER AVE & OLIVE AVE INTERSECTION TRAFFIC SIGNAL AND ROADWAY IMPROVEMENTS(TC)   
129                                                               HEALDSBURG AVENUE BETWEEN POWELL AVENUE AND PASSALAQUA ROAD COMPLETE STREET IMPROVEMENTS   
137  IN AND NEAR GRIDLEY, FROM HOLLIS LANE TO NORTH OF FORD AVENUE. DIST-CO-RTE: 03-BUT-99 PM/PM: R3.1/5.0 REHABILITATE PAVEMENT, UPGRADE TRANSPORTATION M   
143  IN ELK GROVE ON GRANT LINE ROAD FROM SHERMAN OAKS CT. TO ALEILANI LANE. INTERSECTION OPERATIONAL IMPROVEMENTS IN THE FORM OF EITHER SIGNAL MODIFICATI   
144                                      IN FRANKLIN/BEACHWOOD ON BEACHWOOD DRIVE BETWEEN DAN WARD ROAD AND CABOT AVENUE CONSTRUCT PEDESTRIAN IMPROVEMENTS   
148                                                 IN MERCED ON MOTEL DRIVE BETWEEN CAROL AVENUE AND MERCED AVENUE CONSTRUCT PEDESTRIAN/BIKE IMPROVEMENTS   
156  IN STOCKTON, PARALLEL TO MARCH LANE IN THE EAST BAY MUD CORRIDOR BETWEEN BROOKSIDE ROAD AND HILLSBORO WAY. RECONSTRUCT, WIDEN AND IMPROVE EXISTING PA   
159    IN THE CITY OF GALT ON WALNUT AVE, FROM CARILLION TO ELK HILLS DR. OVERLAY THE STREET AND UPGRADE THE BIKE LANES TO HIGH-VISIBILITY GREEN STANDARD.   
172  INTERSECTIONS OF EL DORADO STREET AND ROBINHOOD DRIVE, EL DORADO STREET AND BENJAMIN HOLT DRIVE, AND MARCH LANE AND MCGAW STREET. UPGRADE TRAFFIC SIG   
178                         JENSEN AVE. FROM ELM AVENUE TO TEMPERANCE AVENUE INSTALL ITS ADAPTIVE SYSTEM, UPGRADE DETECTION AND SYNCHRONIZE CORRIDOR. (TC)   
179                                                                                   JENSEN AVENUE FROM DICKERSEN TO MADERA AVENUES SHOULDER IMPROVEMENTS   
192                                                                                        MICHIGAN AVE. BETWEEN 19TH & 20TH STREET BIKE PATH IMPROVEMENTS   
198                                                                 MONTEREY ROAD FROM COTTLE ROAD TO METCALF ROAD UPGRADE GUARDRAILS TO CURRENT STANDARDS   
218  ON MCKEE ROAD BETWEEN ROUTE 101 AND TOYON AVENUE IMPLEMENT SAFETY IMPROVEMENTS INCLUDING MEDIAN ISLANDS, ADA CURB RAMP, SPEED RADAR SIGN, ENHANCED CR   
220                              

In [72]:
text.sample(10)

project_title  \
316   VARIOUS ROAD SEGMENTS THROUGHOUT CALAVERAS COUNTY (EXCLUDING STATE ROUTES AND THE CITY OF ANGELS CAMP) CONDUCT A ROADWAY SAFETY SIGNAGE AUDIT (RSSA)   
228                                                                                      ORWOOD BRIDGE (28C-0024) AT INDIAN SLOUGH BRIDGE REPLACEMENT (TC)   
2         16 LOCATIONS IN THE CITY OF FAIRFIELD INSTALL ADAPTIVE SIGNAL TIMING AND ADVANCED DILEMMA-ZONE DETECTION, AND ENHANCE THE VISIBILITY OF SIGNALS.   
276                                  SPERRY AVENUE FROM AMERICAN EAGLE AVENUE TO WARD AVENUE. ROAD REHABILITATION / RECONSTRUCTION WITH DIG OUTS AS NEEDED   
289  THE INTERSECTIONS OF MILITARY WEST AND WEST 5TH ST. AND HASTINGS DR. AND LONDON DR. PEDESTRIAN CROSSING IMPROVEMENTS, INCLUDING: ADA RAMPS RRFB SYSTE   
218  ON MCKEE ROAD BETWEEN ROUTE 101 AND TOYON AVENUE IMPLEMENT SAFETY IMPROVEMENTS INCLUDING MEDIAN ISLANDS, ADA CURB RAMP, SPEED RADAR SIGN, ENHANCED CR   
4    21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR   
329                                                                  WOODLAKE AT THE INTERSECTION OF SIERRA AVE AND CASTLE ROCK STREET NEW ROUNDABOUT (TC)   
300  VARIOUS  LOCAL STREETS WITHIN THE CITY OF LOS ANGELES TO RE-SURFACE  LOCAL STREETS OF APPROXIMATELY 45 MILE IN TOTAL LENGTH  AND CONSTRUCT ACCESS RAM   
103                                                      FIFTEEN (15) SIGNALIZED INTERSECTIONS THROUGHOUT THE CITY INSTALL ADVANCED DILEMMA ZONE DETECTION   

                                    summary  
316  [COUNTY, STATE, CITY, SAFETY, SIGNAGE]  
228                        [BRIDGE, BRIDGE]  
2          [CITY, INSTALL, SIGNAL, SIGNALS]  
276                        [REHABILITATION]  
289             [INTERSECTIONS, PEDESTRIAN]  
218                                [SAFETY]  
4                   [INTERSECTIONS, SIGNAL]  
329                     [INTERSECTION, NEW]  
300                       [CITY, CONSTRUCT]  
103          [INTERSECTIONS, CITY, INSTALL]

### Function for getting project types using np.where
(str format)

In [74]:
#using subset of full data
proj_sub = proj_all>>select(_.implementing_agency,
                            _.program_code_description,
                            _.project_title,
                            _.program_code,
                            _.improvement_type_description)

In [75]:
proj_sub.sample()

implementing_agency   program_code_description  \
37  Metropolitan Transportation Commission  STBG-URBANIZED >200K IIJA   

                                                                                                                                      project_title  \
37  BAY AREA REGIONAL: REGIONAL PLANNING ACTIVITIES AND PLANNING, PROGRAMMING AND MONITORING (PPM). PRIOR YEAR FUNDING PROGRAMMED ON REG170001 (TC)   

        program_code improvement_type_description  
37  Y230//Y230//Y230          Other//Other//Other

In [ ]:
#sample of function
((utils.add_description(proj_sub, 'project_title'))).head(10)

implementing_agency      program_code_description  \
0           Hawthorne  HIGHWAY SAFETY IMP PROG IIJA   
1             Yucaipa    NATIONAL HIGHWAY PERF IIJA   
2           Fairfield  HIGHWAY SAFETY IMP PROG IIJA   
3         Lake County   STBG IIJA OFF-SYSTEM BRIDGE   
4           Vacaville  HIGHWAY SAFETY IMP PROG IIJA   
5             Benicia  HIGHWAY SAFETY IMP PROG IIJA   
6         Pico Rivera    NATIONAL HIGHWAY PERF IIJA   
7        Marin County     STBG-URBANIZED >200K IIJA   
8        Marin County  HIGHWAY SAFETY IMP PROG IIJA   
9           Los Banos    CONGESTION MITIGATION IIJA   

                                                                                                                                           project_title  \
0         120TH STREET FROM PRAIRIE AVENUE TO FELTON AVENUE. TRAFFIC SIGNAL UPGRADES AT NINE SIGNALIZED INTERSECTIONS AND IMPROVE CROSSINGS AND SIGNAGE.   
1                         13TH STREET OVER WILSON CREEK FROM OAK GLEN ROAD TO KENTUCKY STREET, LWC 00L0017 REPLACE LOW WATER CROSSING WITH 2-LANE BRIDGE   
2       16 LOCATIONS IN THE CITY OF FAIRFIELD INSTALL ADAPTIVE SIGNAL TIMING AND ADVANCED DILEMMA-ZONE DETECTION, AND ENHANCE THE VISIBILITY OF SIGNALS.   
3                                       2.2 MILES NORTH OF STATE ROUTE 20 ON WITTER SPRINGS ROAD OVER COOPER CREEK. BR.# 14C0119 BRIDGE REPLACEMENT (TC)   
4  21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR   
5                                                                        21 LOCATIONS IN CITY OF BENICIA UPGRADE EXISTING GUARDRAILS AND END TREATMENTS.   
6   3/8 MI WEST OF PARAMOUNT BLVDWASHINGTON BLVD OVER RIO HONDO.LOCAL BRIDGE # 53C0471 BRIDGE REPLACEMENT  (DEMOLISH THE EXISTING 6 LANNE SUPERSTRUCTURE   
7  4 AT TERNERS DRIVE @ DRIVEWAY ENTRANCES TO MULTI-FAMILY HOUSING CLOSEST TO DONAHUE, 4 AT TERNERS DRIVE @ TERRACE WAY, 4 AT TERNERS DRIVE @ TERRACE DR   
8  59 TRAFFIC SIGNAL COUNTYWIDE.  JURISDICTIONS INCLUDE CORTE MADERA, FAIRFAX, MILL VALLEY, NOVATO, SAN ANSELMO, SAN RAFAEL, SAUSALITO, MARIN COUNTY, LA   
9               5-MILES STRETCH OF SR-152 SYNCHRONIZATION OF 14 TRAFFIC SIGNALS ON A FIVE MILE STRETCH OF SR-152 WITH INSTALLATION OF CCTV CAMERAS. (TC)   

             program_code  \
0              YS30//YS30   
1                    Y001   
2              YS30//YS30   
3  Y233//Y233//Y233//Y233   
4              YS30//YS30   
5              YS30//YS30   
6                    Y001   
7                    Y230   
8              YS30//YS30   
9              Y400//Y400   

                                                                              improvement_type_description  \
0                                                                         Construction Engineering//Safety   
1                                                                                                    Other   
2                                                                         Construction Engineering//Safety   
3  Bridge Replacement - No Added Capacity//Preliminary Engineering//Right of Way//Construction Engineering   
4                                                                         Construction Engineering//Safety   
5                                                                         Construction Engineering//Safety   
6                                                                                  Preliminary Engineering   
7                                                                                  Preliminary Engineering   
8                                                                         Construction Engineering//Safety   
9                                           Construction Engineering//Traffic Management/Engineering - HOV   

  project_method     project_type           other  
0        Upgrade  Traffic Signals  Other Projects  
1        Replace           Bridge  Other Pro

In [78]:
#how many entries with no tags
len((utils.add_description(proj_sub, 'project_title'))>>filter(_.project_type==('Project')))

97

In [79]:
#entries with no tag
((utils.add_description(proj_sub, 'project_title'))>>filter(_.project_type==('Project'))>>arrange(-_.project_method)).head(40)

implementing_agency  \
34                                       Colton   
269                                 Los Angeles   
281      Orange County Transportation Authority   
93                                       Fresno   
213                                     Oakland   
19                                   Sacramento   
15                                Fresno County   
17                                     Stockton   
96                                    La Puente   
108                                  Sacramento   
165                                     Modesto   
233                                     Benicia   
251                                Redwood City   
320                                      Clovis   
278                                        Lodi   
282      Orange County Transportation Authority   
16                                    Santa Ana   
26                                   Livingston   
36                                       Rialto   
65                                  Bakersfield   
90                             El Dorado County   
118                                     Fremont   
139                              Citrus Heights   
146                                  Livingston   
150                                    Caltrans   
156                                    Stockton   
185                                  South Gate   
208                                      Merced   
216                                  South Gate   
252                                    Caltrans   
276                                   Patterson   
283  Riverside County Transportation Commission   
297                                      Folsom   
300                                 Los Angeles   
7                                  Marin County   
18                                     Caltrans   
32                                     Palmdale   
35                                      Visalia   
37       Metropolitan Transportation Commission   
38                                   Sacramento   

           program_code_description  \
34       NATIONAL HIGHWAY PERF IIJA   
269      NATIONAL HIGHWAY PERF IIJA   
281       STBG-URBANIZED >200K IIJA   
93    TRANSP ALTERNATIVES FLEX IIJA   
213   TRANS ALTERNATIVES >200K IIJA   
19       CONGESTION MITIGATION IIJA   
15    TRANS ALTERNATIVES >200K IIJA   
17     HIGHWAY SAFETY IMP PROG IIJA   
96     HIGHWAY SAFETY IMP PROG IIJA   
108       STBG-URBANIZED >200K IIJA   
165      CONGESTION MITIGATION IIJA   
233  PROJ TO REDUCE PM 2.5 EMI IIJA   
251  SURFAC TRNSP BLK GRTS-FLX IIJA   
320       STBG-URBANIZED >200K IIJA   
278       STBG-URBANIZED >200K IIJA   
282       STBG-URBANIZED >200K IIJA   
16    TRANS ALTERNATIVES >200K IIJA   
26       CONGESTION MITIGATION IIJA   
36     HIGHWAY SAFETY IMP PROG IIJA   
65       CONGESTION MITIGATION IIJA   
90        STBG-URBANIZED >200K IIJA   
118  SURFAC TRNSP BLK GRTS-FLX IIJA   
139      CONGESTION MITIGATION IIJA   
146  PROJ TO REDUCE PM 2.5 EMI IIJA   
150       STBG-URBANIZED >200K IIJA   
156   TRANS ALTERNATIVES >200K IIJA   
185    HIGHWAY SAFETY IMP PROG IIJA   
208      CONGESTION MITIGATION IIJA   
216    HIGHWAY SAFETY IMP PROG IIJA   
252      CONGESTION MITIGATION IIJA   
276  SURFAC TRNSP BLK GRTS-FLX IIJA   
283       STBG-URBANIZED >200K IIJA   
297       STBG-URBANIZED >200K IIJA   
300       STBG-URBANIZED >200K IIJA   
7         STBG-URBANIZED >200K IIJA   
18        STBG-URBANIZED >200K IIJA   
32     HIGHWAY SAFETY IMP PROG IIJA   
35        STBG-URBANIZED >200K IIJA   
37        STBG-URBANIZED >200K IIJA   
38        STBG-URBANIZED >200K IIJA   

                                                                                                                                             project_title  \
34   BARTON ROAD OVERHEAD AT UNION PACIFIC RAILROAD, 0.25 MILE WEST OF I-215, BR. NO. 54C-0379 REPLACE OVERHEAD WITH TWO-LANE ROAD ACROSS DISCONTINUED RAI   
269                 

In [80]:
#checking remaining common words for no project tags
((utils.get_list_of_words(((utils.add_description(proj_sub, 'project_title'))>>filter(_.project_type=='Project')), "project_title")).value_counts()).head(60)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


avenue           51
st               44
ave              39
road             36
city             23
blvd             20
street           18
east             17
construct        17
north            15
rd               15
county           14
intersections    13
south            13
drive            12
along            12
west             11
san              11
area             10
way              10
dr                9
lane              9
tc                9
path              8
bay               8
river             7
parkway           7
regional          6
install           6
redwood           6
trail             6
streets           6
creek             6
widen             6
mile              6
4                 6
state             6
including         6
project           6
2                 6
corridor          6
park              6
class             6
multiuse          5
interstate        5
pacific           5
sacramento        5
alameda           5
la                5
route             5


In [82]:
#sample of some descriptions that got cut off
((utils.add_description(proj_sub, 'project_title')))>>filter(_.project_type=='Project')>>filter(_.project_title.str.contains('AVENUE'))

implementing_agency  \
15                                    Fresno County   
32                                         Palmdale   
36                                           Rialto   
40                                          Belmont   
67                                Sacramento County   
87                                      Morgan Hill   
92                                        Fairfield   
93                                           Fresno   
122                                          Fresno   
130                                      Santa Rosa   
132                                Twentynine Palms   
134                                        Caltrans   
135  San Bernardino County Transportation Authority   
140                                       Sunnyvale   
141                                El Dorado County   
153            San Diego Association of Governments   
165                                         Modesto   
169                              Los Angeles County   
251                                    Redwood City   
276                                       Patterson   
279                                      Sebastopol   
320                                          Clovis   

           program_code_description  \
15    TRANS ALTERNATIVES >200K IIJA   
32     HIGHWAY SAFETY IMP PROG IIJA   
36     HIGHWAY SAFETY IMP PROG IIJA   
40        STBG-URBANIZED >200K IIJA   
67     HIGHWAY SAFETY IMP PROG IIJA   
87        STBG-URBANIZED >200K IIJA   
92   PROJ TO REDUCE PM 2.5 EMI IIJA   
93    TRANSP ALTERNATIVES FLEX IIJA   
122    HIGHWAY SAFETY IMP PROG IIJA   
130       STBG-URBANIZED >200K IIJA   
132    HIGHWAY SAFETY IMP PROG IIJA   
134       STBG-URBANIZED >200K IIJA   
135       STBG-URBANIZED >200K IIJA   
140  PROJ TO REDUCE PM 2.5 EMI IIJA   
141       STBG-URBANIZED >200K IIJA   
153       STBG-URBANIZED >200K IIJA   
165      CONGESTION MITIGATION IIJA   
169    HIGHWAY SAFETY IMP PROG IIJA   
251  SURFAC TRNSP BLK GRTS-FLX IIJA   
276  SURFAC TRNSP BLK GRTS-FLX IIJA   
279  SURFAC TRNSP BLK GRTS-FLX IIJA   
320       STBG-URBANIZED >200K IIJA   

                                                                                                                                             project_title  \
15   ALONG GROVE AVENUE FROM PROSPECT AVENUE TO VALENTINE AVENUE, AND ALONG VALENTINE AVENUE FROM GROVE AVENUE TO NORTH AVENUE INSTALL AN ASPHALT CONCRETE   
32     AVENUE S-8 AND 40TH STREET EAST. CONVERT AN EXISTING FOUR WAY STOP TO A ROUDABOUT AT THE INTERSECTION OF AVENUE S-8 AND 40TH STREET EAST..   ORIGIO   
36                                                BASELINE ROAD - WILLOW AVENUE TO MERIDIAN AVENUE CONSTRUCT A 12' WIDE RAISED CENTER MEDIAN AND ADA RAMP.   
40   BELMONT: CHULA VISTA FROM ALAMEDA DE LAS PULGAS TO RALSTON AVE, 6TH AVENUE FROM RALSTON AVENUE TO HILL STREET, 6TH AVENUE FROM EMMETT AVENUE TO HARBO   
67    COYLE AVENUE FROM WOODLEIGH TO DEWEY, HILLSDALE BOULEVARD FROM FRIZELL AVENUE TO MCCLOUD DRIVE, AND WATT AVENUE FROM ELKHORN BOULEVARD TO 700' NORTH   
87   EAST DUNNE AVENUE BETWEEN LOWER THOMAS GRADE AND THE EASTERN CITY LIMIT. FEDERAL PARTICIPATING LOCATION SEGMENTS: 1) LOWER THOMAS GRADE TO FLAMING OA   
92   EAST TABOR AVENUE ( BETWEEN DOVER  AVENUE AND CLAY BANK ROAD) AND SUNSET AVENUE ( BETWEEN EAST TABOR AVENUE AND TRAVIS BOULEVARD) IMPLEMENT PEDESTRIA   
93   EASTBOUND MCKINLEY FROM MILLBROOK AVENUE TO CLOVIS AVENUE ALONG THE NORTH BANK OF THE MILL DITCH CANAL. PAVED PATH, LIGHTING, BENCHES,FENCING, IRRIGA   
122  FRESNO STREET AND THOMAS AVENUE INTERSECTIONS, FRESNO STREET AND SAN JOSE AVENUE INTERSECTION, FRESNO AND R STREETS INTERSECTION, FRESNO STREET AND C   
130   HEARN AVENUE BETWEEN STONY POINT ROAD, DUTTON AVENUE AND RANGE AVENUE BETWEEN GUERNEVILLE ROAD AND JENNINGS AVENUE, AND RANGE AVENUE BETWEEN RUSSELL   
132  HIGH-SPEED RURAL RESIDENTIAL AREAS BORDERED BY TWO MILE ROAD ON THE NORTH, BASELINE ROAD ON THE SOUTH, ENCELIA DRIVE TO THE W

In [83]:
## sample of what third colum, `other` captures
((utils.add_description(proj_sub, 'project_title')))>>filter(_.other!='Other Projects')


implementing_agency        program_code_description  \
27                         El Dorado County       STBG-URBANIZED >200K IIJA   
29                                 Palmdale       STBG-URBANIZED >200K IIJA   
39                               Bellflower    HIGHWAY SAFETY IMP PROG IIJA   
57                                  Alameda   TRANS ALTERNATIVES >200K IIJA   
81                              Los Angeles   TRANSP ALTERNATIVES FLEX IIJA   
85                                    Chico      CONGESTION MITIGATION IIJA   
95                                   Fresno   TRANS ALTERNATIVES >200K IIJA   
119                                 Fremont  SURFAC TRNSP BLK GRTS-FLX IIJA   
120  Metropolitan Transportation Commission       STBG-URBANIZED >200K IIJA   
142                        El Dorado County      CONGESTION MITIGATION IIJA   
148                                  Merced  PROJ TO REDUCE PM 2.5 EMI IIJA   
159                                    Galt       STBG-URBANIZED >200K IIJA   
192                            Santa Monica   TRANS ALTERNATIVES >200K IIJA   
204                                  Clovis       STBG-URBANIZED >200K IIJA   
207                             Yuba County       STBG-URBANIZED >200K IIJA   
210                                  Merced  PROJ TO REDUCE PM 2.5 EMI IIJA   
231                                   Hemet   TRANSP ALTERNATIVES FLEX IIJA   
236                               Tehachapi      CONGESTION MITIGATION IIJA   
238                             Kern County   TRANS ALTERNATIVES >200K IIJA   
241                       Sacramento County    HIGHWAY SAFETY IMP PROG IIJA   
254                                 Vallejo       STBG-URBANIZED >200K IIJA   
280                                Caltrans      CONGESTION MITIGATION IIJA   
293                                  Fresno      CONGESTION MITIGATION IIJA   
298           Tahoe Transportation District       STBG-URBANIZED >200K IIJA   
314                                 Windsor  SURFAC TRNSP BLK GRTS-FLX IIJA   

                                                                                                                                             project_title  \
27                       AT MISSOURI FLAT RD 0.8 MILES SOUTH OF GOLDEN CENTER DRIVE CONSTRUCT BIKE TRAIL CONNECTION, EXTEND CURB, GUTTER, AND SIDEWALK. TC   
29         AVE R  BETWEEN SIERRA HIGHWAY AND 25TH STREET. AVE R COMPLETE STREET WITH SIDEWALKS GAP CLOSURES, BIKE LANES, ADA RAMPS AND ENHANCED CROSSWALKS   
39   BELLFLOWER BOULEVARD FROM ARTESIA BOULEVARD TO SOUTH CITY LIMITS (350' SOUTH OF ROSE STREET). INSTALLING CLASS II BIKE LANES, RAISED MEDIAN, BULB-OUT   
57    CENTRAL BETWEEN PACIFIC AVENUE/MAIN STREET AND SHERMAN STREET/ENCINAL AVENUE. REDUCE ROADWAY FROM 4 TO 3 LANES FOR BIKE LANES AND SEPARATED BIKEWAY,   
81    DOWNTOWN LOS ANGELES, BROADWAY BETWEEN 4TH AND 6TH STREETS PEDESTRIAN SAFETY IMPROVEMENTS INCLUDING CURB EXTENSIONS, WIDENED SIDEWALK, CROSSWALK AND   
85                                                        E. 20TH STREET FROM THE MALL TO THE END OF BUSINESS LANE (ADJACENT TO SR99) BIKEWAY IMPROVEMENTS   
95   EASTSIDE OF BARTON AVENUE FROM CHURCH TO FLORENCE AND FLORENCE AVENUE FROM BARTON TO APPROXIMATELY 105 FT WEST OF JACKSON. INSTALL SIDEWALK, CURB RAM   
119                                      FREMONT BLVD. AND WALNUT AVE. INTERSECTION CONNECT TO EXISTING ELEVATED/SEPARATED CLASS IV BIKEWAYS AND SIDEWALKS   
120                                            FREMONT, RICHMOND, AND MARIN AND SONOMA COUNTIES, ALONG THE SMART CORRIDOR. BIKE SHARE CAPITAL PROGRAM (TC)   
142  IN EL DORADO COUNTY, IN THE COMMUNITY OF POLLOCK PINES, PONY EXPRESS TRAIL, FROM SANDERS DRIVE TO SLY PARK ROAD. INSTALL 1.7 MILES OF CLASS 2 BIKE LA   
148                                                 IN MERCED ON MOTEL DRIVE BETWEEN CAROL AVENUE AND MERCED AVENUE CONSTRUCT PEDESTRIAN/BIKE IMPROVEMENTS   
159    IN THE CITY OF GALT ON WALNUT AVE, FROM CARILLION TO ELK HILLS DR.

## Combine for Public friendly title

In [85]:
title_new = (utils.add_description(proj_sub, 'project_title'))

In [86]:
title_new.head()

implementing_agency      program_code_description  \
0           Hawthorne  HIGHWAY SAFETY IMP PROG IIJA   
1             Yucaipa    NATIONAL HIGHWAY PERF IIJA   
2           Fairfield  HIGHWAY SAFETY IMP PROG IIJA   
3         Lake County   STBG IIJA OFF-SYSTEM BRIDGE   
4           Vacaville  HIGHWAY SAFETY IMP PROG IIJA   

                                                                                                                                           project_title  \
0         120TH STREET FROM PRAIRIE AVENUE TO FELTON AVENUE. TRAFFIC SIGNAL UPGRADES AT NINE SIGNALIZED INTERSECTIONS AND IMPROVE CROSSINGS AND SIGNAGE.   
1                         13TH STREET OVER WILSON CREEK FROM OAK GLEN ROAD TO KENTUCKY STREET, LWC 00L0017 REPLACE LOW WATER CROSSING WITH 2-LANE BRIDGE   
2       16 LOCATIONS IN THE CITY OF FAIRFIELD INSTALL ADAPTIVE SIGNAL TIMING AND ADVANCED DILEMMA-ZONE DETECTION, AND ENHANCE THE VISIBILITY OF SIGNALS.   
3                                       2.2 MILES NORTH OF STATE ROUTE 20 ON WITTER SPRINGS ROAD OVER COOPER CREEK. BR.# 14C0119 BRIDGE REPLACEMENT (TC)   
4  21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR   

             program_code  \
0              YS30//YS30   
1                    Y001   
2              YS30//YS30   
3  Y233//Y233//Y233//Y233   
4              YS30//YS30   

                                                                              improvement_type_description  \
0                                                                         Construction Engineering//Safety   
1                                                                                                    Other   
2                                                                         Construction Engineering//Safety   
3  Bridge Replacement - No Added Capacity//Preliminary Engineering//Right of Way//Construction Engineering   
4                                                                         Construction Engineering//Safety   

  project_method     project_type           other  
0        Upgrade  Traffic Signals  Other Projects  
1        Replace           Bridge  Other Projects  
2        Install          Signals  Other Projects  
3        Replace           Bridge  Other Projects  
4        Improve          Signals  Other Projects

In [87]:
#replacing with function
# title_new['project_name_new'] = title_new["project_method"] + " " + title_new["project_type"] + " in " + title_new["implementing_agency"]

In [88]:
title_new = utils.add_new_title(title_new, 'project_method', 'project_type', 'implementing_agency')

In [89]:
title_new.head()

implementing_agency      program_code_description  \
0           Hawthorne  HIGHWAY SAFETY IMP PROG IIJA   
1             Yucaipa    NATIONAL HIGHWAY PERF IIJA   
2           Fairfield  HIGHWAY SAFETY IMP PROG IIJA   
3         Lake County   STBG IIJA OFF-SYSTEM BRIDGE   
4           Vacaville  HIGHWAY SAFETY IMP PROG IIJA   

                                                                                                                                           project_title  \
0         120TH STREET FROM PRAIRIE AVENUE TO FELTON AVENUE. TRAFFIC SIGNAL UPGRADES AT NINE SIGNALIZED INTERSECTIONS AND IMPROVE CROSSINGS AND SIGNAGE.   
1                         13TH STREET OVER WILSON CREEK FROM OAK GLEN ROAD TO KENTUCKY STREET, LWC 00L0017 REPLACE LOW WATER CROSSING WITH 2-LANE BRIDGE   
2       16 LOCATIONS IN THE CITY OF FAIRFIELD INSTALL ADAPTIVE SIGNAL TIMING AND ADVANCED DILEMMA-ZONE DETECTION, AND ENHANCE THE VISIBILITY OF SIGNALS.   
3                                       2.2 MILES NORTH OF STATE ROUTE 20 ON WITTER SPRINGS ROAD OVER COOPER CREEK. BR.# 14C0119 BRIDGE REPLACEMENT (TC)   
4  21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR   

             program_code  \
0              YS30//YS30   
1                    Y001   
2              YS30//YS30   
3  Y233//Y233//Y233//Y233   
4              YS30//YS30   

                                                                              improvement_type_description  \
0                                                                         Construction Engineering//Safety   
1                                                                                                    Other   
2                                                                         Construction Engineering//Safety   
3  Bridge Replacement - No Added Capacity//Preliminary Engineering//Right of Way//Construction Engineering   
4                                                                         Construction Engineering//Safety   

  project_method     project_type           other  \
0        Upgrade  Traffic Signals  Other Projects   
1        Replace           Bridge  Other Projects   
2        Install          Signals  Other Projects   
3        Replace           Bridge  Other Projects   
4        Improve          Signals  Other Projects   

                       project_name_new  
0  Upgrade Traffic Signals in Hawthorne  
1             Replace Bridge in Yucaipa  
2          Install Signals in Fairfield  
3         Replace Bridge in Lake County  
4          Improve Signals in Vacaville

### Export Sample Data with all columns

In [91]:
test_df = (utils.add_description(proj_all, 'project_title'))

In [92]:
test_df = utils.add_new_title(test_df, 'project_method', 'project_type', 'implementing_agency')

In [93]:
test_df = utils.title_column_names(test_df)

#### Export progress

In [94]:
# test_df.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_wip2.csv")

### Using another column `improvement_type_description`

In [95]:
def update_no_matched(df, flag_col, desc_col):
    """
    function to itreate over projects that did not match the first time
    using an existing project's short description of project type. 
    """
    
    if df[flag_col] == "Project":
        return 
    
    
    return df

In [96]:
no_match = ((utils.add_description(proj_sub, 'project_title'))>>filter(_.project_type==('Project')))

In [97]:
## using improvement type description
clean_type_list_df = utils.get_list_of_words(no_match, "improvement_type_description") 


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [98]:
(clean_type_list_df.value_counts()).head(30)

construction                     34
engineering                      28
4r                               23
pedestrians                      20
bicycles                         18
maintenance                      14
engineeringsafety                13
preliminary                      12
restoration                      12
engineeringfacilities            11
added                             9
resurfacing4r                     9
safety                            8
facilities                        8
engineeringsafetyconstruction     8
rehabilitationconstruction        7
rehabilitation                    5
capacity                          5
roadwayconstruction               4
capacityconstruction              4
new                               4
engineeringpreliminary            3
resurfacingconstruction           3
roadway                           3
engineeringnew                    3
way                               3
right                             3
otherotherother             

In [99]:
no_match.head()

implementing_agency       program_code_description  \
7         Marin County      STBG-URBANIZED >200K IIJA   
15       Fresno County  TRANS ALTERNATIVES >200K IIJA   
16           Santa Ana  TRANS ALTERNATIVES >200K IIJA   
17            Stockton   HIGHWAY SAFETY IMP PROG IIJA   
18            Caltrans      STBG-URBANIZED >200K IIJA   

                                                                                                                                            project_title  \
7   4 AT TERNERS DRIVE @ DRIVEWAY ENTRANCES TO MULTI-FAMILY HOUSING CLOSEST TO DONAHUE, 4 AT TERNERS DRIVE @ TERRACE WAY, 4 AT TERNERS DRIVE @ TERRACE DR   
15  ALONG GROVE AVENUE FROM PROSPECT AVENUE TO VALENTINE AVENUE, AND ALONG VALENTINE AVENUE FROM GROVE AVENUE TO NORTH AVENUE INSTALL AN ASPHALT CONCRETE   
16  ALONG NEIGHBORHOOD STREETS ADJACENT TO DAVIS ELEMENTARY SCHOOL FROM 14TH STREET TO 15TH STREET. CONSTRUCT BULB-OUT AT CROSSINGS NEAR DAVIS ELEMENTARY   
17                                                          ALONG PACIFIC AVE. FROM CALAVERAS RIVER TRAIL TO WEST HAMMER LANE. INSTALL RAISED MEDIAN CURB   
18  ALONG THE I-880 CORRIDOR SF BAY AREA: REGIONWIDE: IMPLEMENT A COLLECTIVE APPROACH TO FREEWAY OPERATIONS AND MANAGEMENT, INCLUDING COMMUNICATIONS NETW   

   program_code  \
7          Y230   
15   Y301//Y301   
16         Y301   
17         YS30   
18         Y230   

                                         improvement_type_description  \
7                                             Preliminary Engineering   
15  Construction Engineering//Facilities for Pedestrians and Bicycles   
16                                           Construction Engineering   
17                                                             Safety   
18                                            Preliminary Engineering   

   project_method project_type           other  \
7                      Project  Other Projects   
15        Install      Project  Other Projects   
16      Construct      Project  Other Projects   
17        Install      Project  Other Projects   
18                     Project  Other Projects   

                    project_name_new  
7            Project in Marin County  
15  Install Project in Fresno County  
16    Construct Project in Santa Ana  
17       Install Project in Stockton  
18               Project in Caltrans

In [100]:
no_match>>count(_.improvement_type_description)>>filter(_.n>1)>>arrange(-_.n)

improvement_type_description   n
17                                   Construction Engineering//Safety  10
29                                            Preliminary Engineering   9
12  Construction Engineering//Facilities for Pedestrians and Bicycles   8
21                            Facilities for Pedestrians and Bicycles   8
35                                                             Safety   7
9         4R - Restoration & Rehabilitation//Construction Engineering   6
25                                                              Other   6
7                                   4R - Restoration & Rehabilitation   4
3             4R - Maintenance  Resurfacing//Construction Engineering   3
4                                              4R - No Added Capacity   3
0                                                 4R - Added Capacity   2
11                                           Construction Engineering   2
22                                          New  Construction Roadway   2
26                                                Other//Other//Other   2
30                   Preliminary Engineering//Preliminary Engineering   2

In [101]:
no_match>>filter(_.improvement_type_description.str.contains('Pedestrians and Bicycles'))

implementing_agency  \
15                                              Fresno County   
19                                                 Sacramento   
26                                                 Livingston   
65                                                Bakersfield   
82                                                Los Angeles   
90                                           El Dorado County   
92                                                  Fairfield   
93                                                     Fresno   
139                                            Citrus Heights   
146                                                Livingston   
156                                                  Stockton   
213                                                   Oakland   
240                      Napa Valley Transportation Authority   
251                                              Redwood City   
258                                          San Mateo County   
274                                                Sacramento   
279                                                Sebastopol   
295  Los Angeles County Metropolitan Transportation Authority   

           program_code_description  \
15    TRANS ALTERNATIVES >200K IIJA   
19       CONGESTION MITIGATION IIJA   
26       CONGESTION MITIGATION IIJA   
65       CONGESTION MITIGATION IIJA   
82    TRANS ALTERNATIVES >200K IIJA   
90        STBG-URBANIZED >200K IIJA   
92   PROJ TO REDUCE PM 2.5 EMI IIJA   
93    TRANSP ALTERNATIVES FLEX IIJA   
139      CONGESTION MITIGATION IIJA   
146  PROJ TO REDUCE PM 2.5 EMI IIJA   
156   TRANS ALTERNATIVES >200K IIJA   
213   TRANS ALTERNATIVES >200K IIJA   
240  SURFAC TRNSP BLK GRTS-FLX IIJA   
251  SURFAC TRNSP BLK GRTS-FLX IIJA   
258  SURFAC TRNSP BLK GRTS-FLX IIJA   
274      CONGESTION MITIGATION IIJA   
279  SURFAC TRNSP BLK GRTS-FLX IIJA   
295   TRANSP ALTERNATIVES FLEX IIJA   

                                                                                                                                             project_title  \
15   ALONG GROVE AVENUE FROM PROSPECT AVENUE TO VALENTINE AVENUE, AND ALONG VALENTINE AVENUE FROM GROVE AVENUE TO NORTH AVENUE INSTALL AN ASPHALT CONCRETE   
19   ALONG UNUSED RAIL CORRIDOR NEAR FREEPORT BLVD FROM SUTTERVILLE ROAD TO SOUTH OF MEADOWVIEW/POCKET ROAD (4.5 MILES) RAILS TO TRAILS PROJECT ; NEW CLAS   
26                                                                   AT MAX FOSTER SPORTS COMPLEX AREA IN LIVINGSTON CONSTRUCT MULTI-USE PATH IMPROVEMENTS   
65   CONSTRUCT A MULTI-USE PATH (6 MILES) ALONG THE FRIANT-KERN CANAL IN BAKERSFIELD: BOUNDED BY 7TH STANDARD ROAD AND KERN RIVER PARKWAY. CONSTRUCT CLASS   
82   DTLA ARTS DISTRICT AREA FROM 2ND STREET ON THE NORTH, 7TH STREET ON THE SOUTH, ALAMEDA STREET ON THE WEST AND THE LA RIVER ON THE EAST WITH A FOCUS O   
90   EAST SAN BERNARDINO AVE FROM WEST OF UPPER TRUCKEE RIVER TO TAHOE PARADISE PARK IN THE COMMUNITY OF MYERS CONSTRUCT APPROXIMATELY 0.37 MILES OF CLASS   
92   EAST TABOR AVENUE ( BETWEEN DOVER  AVENUE AND CLAY BANK ROAD) AND SUNSET AVENUE ( BETWEEN EAST TABOR AVENUE AND TRAVIS BOULEVARD) IMPLEMENT PEDESTRIA   
93   EASTBOUND MCKINLEY FROM MILLBROOK AVENUE TO CLOVIS AVENUE ALONG THE NORTH BANK OF THE MILL DITCH CANAL. PAVED PATH, LIGHTING, BENCHES,FENCING, IRRIGA   
139                           IN CITRUS HEIGHTS, BETWEEN ARCADE CREEK PARK PRESERVE AND WACHTEL WAY. CONSTRUCT A 2.9 MILE LONG CLASS 1 MULTI-USE TRAIL. TC   
146  IN LIVINGSTON, CONSTRUCT PHASE 2 MAX FOSTER MULTIUSE PATH (ALONG DWIGHT WAY, BETWEEN NORTH TRIGGER LN & TEHAMA DR,) CONSTRUCT A CONCRETE MULTIUSE PAT   
156  IN STOCKTON, PARALLEL TO MARCH LANE IN THE EAST BAY MUD CORRIDOR BETWEEN BROOKSIDE ROAD AND HILLSBORO WAY. RECONSTRUCT, WIDEN AND IMPROVE EXISTING PA   
213  OAKLAND - 14TH STREET SAFE ROUTES IN THE CITY. ON 14TH ST BETWEEN BRUSH ST AND OAK ST REDUCE TRAVEL LANES FROM 4 TO 2, ADD PAVED CLASS IV PROTECTED B   
240  POST 

In [102]:
no_match['improvement_type_description'] = no_match['improvement_type_description'].str.upper()

/tmp/ipykernel_2261/1491882977.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [103]:
test = utils.add_description(no_match, 'improvement_type_description')

/home/jovyan/data-analyses/dla/agency_mapping/utils.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/dla/agency_mapping/utils.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/dla/agency_mapping/utils.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [104]:
test.head()

implementing_agency       program_code_description  \
7         Marin County      STBG-URBANIZED >200K IIJA   
15       Fresno County  TRANS ALTERNATIVES >200K IIJA   
16           Santa Ana  TRANS ALTERNATIVES >200K IIJA   
17            Stockton   HIGHWAY SAFETY IMP PROG IIJA   
18            Caltrans      STBG-URBANIZED >200K IIJA   

                                                                                                                                            project_title  \
7   4 AT TERNERS DRIVE @ DRIVEWAY ENTRANCES TO MULTI-FAMILY HOUSING CLOSEST TO DONAHUE, 4 AT TERNERS DRIVE @ TERRACE WAY, 4 AT TERNERS DRIVE @ TERRACE DR   
15  ALONG GROVE AVENUE FROM PROSPECT AVENUE TO VALENTINE AVENUE, AND ALONG VALENTINE AVENUE FROM GROVE AVENUE TO NORTH AVENUE INSTALL AN ASPHALT CONCRETE   
16  ALONG NEIGHBORHOOD STREETS ADJACENT TO DAVIS ELEMENTARY SCHOOL FROM 14TH STREET TO 15TH STREET. CONSTRUCT BULB-OUT AT CROSSINGS NEAR DAVIS ELEMENTARY   
17                                                          ALONG PACIFIC AVE. FROM CALAVERAS RIVER TRAIL TO WEST HAMMER LANE. INSTALL RAISED MEDIAN CURB   
18  ALONG THE I-880 CORRIDOR SF BAY AREA: REGIONWIDE: IMPLEMENT A COLLECTIVE APPROACH TO FREEWAY OPERATIONS AND MANAGEMENT, INCLUDING COMMUNICATIONS NETW   

   program_code  \
7          Y230   
15   Y301//Y301   
16         Y301   
17         YS30   
18         Y230   

                                         improvement_type_description  \
7                                             PRELIMINARY ENGINEERING   
15  CONSTRUCTION ENGINEERING//FACILITIES FOR PEDESTRIANS AND BICYCLES   
16                                           CONSTRUCTION ENGINEERING   
17                                                             SAFETY   
18                                            PRELIMINARY ENGINEERING   

   project_method                    project_type           other  \
7                                         Project  Other Projects   
15      Construct  Pedestrian Safety Improvements  Other Projects   
16      Construct                         Project  Other Projects   
17                                        Project  Other Projects   
18                                        Project  Other Projects   

                    project_name_new  
7            Project in Marin County  
15  Install Project in Fresno County  
16    Construct Project in Santa Ana  
17       Install Project in Stockton  
18               Project in Caltrans